# **Notebook From https://github.com/TheLastBen/fast-stable-diffusion, if you encounter any issues, feel free to discuss them.** [Support](https://ko-fi.com/thelastben)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown # Install/Update AUTOMATIC1111 repo
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time

if not os.path.exists("/content/gdrive/MyDrive/"):
  print('[1;31mGdrive not connected, using colab storage ...')
  time.sleep(4)
  !mkdir -p /content/gdrive/MyDrive/
with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  %mkdir -p /content/gdrive/MyDrive/sd
  %cd /content/gdrive/MyDrive/sd
  !$fgitclone -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-webui
  !mkdir -p /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/huggingface
  !ln -s /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/huggingface /root/.cache/

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/
  !git reset --hard
  time.sleep(1)
  !rm webui.sh
print('[1;32m')
!git pull


def download_from_civitai(model_subdirectory: str, directory_path: str):
    url = f"https://civitai.com/api/download/models/{model_subdirectory}"
    download_file(url, directory_path)


def download_from_huggingfaces(repository: str, file: str, directory_path: str) -> None:
    url = f"https://huggingface.co/{repository}/resolve/main/{file}"
    download_file(url, directory_path, file)


def download_from_github(repository: str, directory_path: str):
    url = f"https://github.com/{repository}"
    path = f"{directory_path}/{repository.split('/')[1]}"
    !git clone {url} {path} --depth 1


def download_file(url, directory_path, file=""):
    !apt-get -y install -qq aria2
    if file:
        !aria2c -c -x 10 -s 10 -k 1M {url} -d {directory_path} -o {file}
    else:
        !aria2c -c -x 10 -s 10 -k 1M {url} -d {directory_path} --show-console-readout false
    clear_output()
    print(f'[1;32m{file if file else "File"} downloaded.')

clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # Requirements

print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /content/ 
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  if not os.path.exists('/content/gdrive/MyDrive/sd/stablediffusion'):
    !tar -C / --zstd -xf sd.tar.zst 
  !tar -C / --zstd -xf A1111_dep.tar.zst
  !rm *.deb | rm *.zst | rm *.txt

clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # Model Download/Load
Model_Version = "1.5" #@param [ "1.5", "v1.5 Inpainting ", "V2.1-512px", "V2.1-768px"]

Redownload_the_original_model = True #@param {type:"boolean"}

if Redownload_the_original_model:
  with capture.capture_output() as cap:
    def rmv():
      !wget -q -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/precompiled/README.md
      !mv /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/trashfile.f
      time.sleep(2)
      !rm /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/trashfile.f
      clear_output()
    rmv()

#@markdown Or
Path_to_MODEL = "" #@param {type:"string"}
#@markdown - Insert the full path of your trained model or to a folder containing multiple models.

#@markdown Or
MODEL_LINK = "https://huggingface.co/m4gnett/any-pastel/resolve/main/AnyPastel-hard-fp16.safetensors" #@param {type:"string"}
safetensors = True #@param {type:"boolean"}
Use_temp_storage = False #@param {type:"boolean"}



def newmdl():

    if not os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
      %cd /content/
      clear_output()
      !git init
      !git lfs install --system --skip-repo
      !$fgitclone --branch fp16 "https://huggingface.co/runwayml/stable-diffusion-v1-5"
      if os.path.exists('/content/stable-diffusion-v1-5'):
        !$fgitclone "https://huggingface.co/stabilityai/sd-vae-ft-mse"
        !rm -r /content/stable-diffusion-v1-5/vae
        !mv /content/sd-vae-ft-mse /content/stable-diffusion-v1-5/vae
        !wget -O convertosd.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosd.py
        !sed -i '201s@.*@    model_path = "/content/stable-diffusion-v1-5"@' /content/convertosd.py
        !sed -i '202s@.*@    checkpoint_path= "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt"@' /content/convertosd.py
        clear_output()       
        !python /content/convertosd.py
        !rm /content/convertosd.py
        if os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
          model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
          clear_output()
          inf('\u2714 Done','success', '50px')
        else:
          inf('\u2718 Something went wrong, try again','danger', "250px")
      else:
        inf('\u2718 Something went wrong','danger', "200px")

    else:
      model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
      clear_output()
      inf('\u2714 Model already exists, check the box "Redownload_the_original_model" to redownload/download the V1.5','primary', '700px')

    if os.path.exists('/content/.git'):
      !rm -r /content/.git

    if os.path.exists('/content/stable-diffusion-v1-5'):
      !rm -r /content/stable-diffusion-v1-5

    return model

def V2():

    if not os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
      %cd /content/
      clear_output()
      !mkdir "/content/stable-diffusion-V2"
      %cd "/content/stable-diffusion-V2"
      !git init
      !git lfs install --system --skip-repo
      if Model_Version == "V2.1-768px":
        !git remote add -f origin  "https://huggingface.co/stabilityai/stable-diffusion-2-1"
      elif Model_Version == "V2.1-512px":
        !git remote add -f origin  "https://huggingface.co/stabilityai/stable-diffusion-2-1-base"
      !git config core.sparsecheckout true
      !echo -e "scheduler\ntext_encoder\ntokenizer\nunet\nvae\nfeature_extractor\nmodel_index.json\n!*.safetensors" > .git/info/sparse-checkout
      !git pull origin fp16
      %cd /content
      !wget -O convertosdv2.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosdv2.py
      clear_output()
      !python /content/convertosdv2.py --fp16 /content/stable-diffusion-V2 /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt
      !rm /content/convertosdv2.py
      if os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
        model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
        clear_output()
        inf('\u2714 Done','success', '50px')
      else:
        inf('\u2718 Something went wrong, try again','danger', "250px")

    else:
      model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
      clear_output()
      inf('\u2714 Model already exists, check the box "Redownload_the_original_model" to redownload/download the V2','primary', '700px')

    if os.path.exists('/content/.git'):
      !rm -r /content/.git
      !rm -r /content/convertosdv2.py
    if os.path.exists('/content/stable-diffusion-V2'):
      !rm -r /content/stable-diffusion-V2

    return model

def inpmdl():

    if not os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'):
      %cd /content/
      clear_output()
      !git init
      !git lfs install --system --skip-repo
      !$fgitclone --branch fp16 "https://huggingface.co/runwayml/stable-diffusion-inpainting"
      if os.path.exists('/content/stable-diffusion-inpainting'):
        !$fgitclone "https://huggingface.co/stabilityai/sd-vae-ft-mse"
        !rm -r /content/stable-diffusion-inpainting/vae
        !mv /content/sd-vae-ft-mse /content/stable-diffusion-inpainting/vae        
        !wget -O convertosd.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosd.py
        !sed -i '201s@.*@    model_path = "/content/stable-diffusion-inpainting"@' /content/convertosd.py
        !sed -i '202s@.*@    checkpoint_path= "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt"@' /content/convertosd.py
        clear_output()       
        !python /content/convertosd.py
        !rm /content/convertosd.py
        if os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'):
          model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'
          clear_output()
          inf('\u2714 Done','success', '50px')
        else:
          inf('\u2718 Something went wrong, try again','danger', "250px")
      else:
        inf('\u2718 Something went wrong','danger', "200px")


    else:
      model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'
      clear_output()
      inf('\u2714 inpainting model already exists','primary', '250px')

    if os.path.exists('/content/.git'):
      !rm -r /content/.git

    if os.path.exists('/content/stable-diffusion-inpainting'):
      !rm -r /content/stable-diffusion-inpainting

    return model

if (Path_to_MODEL !=''):
  if os.path.exists(str(Path_to_MODEL)):
    inf('\u2714 Using the trained model.','success', '200px')

  else:
      while not os.path.exists(str(Path_to_MODEL)):
        inf('\u2718 Wrong path, use the colab file explorer to copy the path : ','danger', "400px")
        Path_to_MODEL=input()
      if os.path.exists(str(Path_to_MODEL)):
        inf('\u2714 Using the trained model.','success', '200px')

  model=Path_to_MODEL

elif MODEL_LINK != "":
  gdrv="/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion"
  tmp="/content"
  pth=tmp if Use_temp_storage else gdrv
  %cd $pth
  clear_output()
  if not safetensors:
    modelname="model.ckpt"
  else:
    modelname="model.safetensors"
  !gdown --fuzzy -O $modelname $MODEL_LINK
  if os.path.exists(f'{pth}/{modelname}') and os.path.getsize(f'{pth}/{modelname}') > 1810671599:    
      model=f'{pth}/{modelname}'
      clear_output()
      inf('\u2714 Model downloaded, using the trained model.','success', '350px')
  else:
    if Use_temp_storage:
      !rm $pth/$modelname
    else:
      rmv()
    inf('\u2718 Wrong link, check that the link is valid','danger', "300px")


elif Model_Version=="1.5":
  model=newmdl()

elif Model_Version=="V2.1-512px" or Model_Version=="V2.1-768px":
  model=V2()

else:
   model=inpmdl()

In [ ]:
from time import sleep
#@title <font size="5">Download Models</font>
def get_models() -> None:
    model_1 = "AbyssOrangeMix2_hard"  # @param ["AbyssOrangeMix2_hard", "Anything v3", "Babes", "ComicsBlend", "dalcefo_v3_pastelMix", "Dreamlike Photoreal", "dreamlikesamkuvshinov", "DreamShaper_v3", "F222", "GTM UltimateBlend v3", "HassanBlend v1.5", "Hunter69 v1", "Inkpunk Diffusion v2", "Kotosmix", "NeverEnding Dream", "Novel Inkpunk F222", "openjourney v2", "RPG", "Realistic Vision V1.2", "Samdoesarts Ultmerge", "SD v1.5 inpainting", "SD v1.5", "seek-art-mega-midjrnyv4-dreamlike", "StablyAiffused's Aesthetic v2.6", "SynthwavePunk v2", "SynthwavePunk v3", "Uber Realistic Porn Merge v1.2", "s1dlxBrew"]
    model_2 = "Babes"  # @param ["None", "AbyssOrangeMix2_hard", "Anything v3", "Babes", "ComicsBlend", "dalcefo_v3_pastelMix", "Dreamlike Photoreal", "dreamlikesamkuvshinov", "DreamShaper_v3", "F222", "GTM UltimateBlend v3", "HassanBlend v1.5", "Hunter69 v1", "Inkpunk Diffusion v2", "Kotosmix", "NeverEnding Dream", "Novel Inkpunk F222", "openjourney v2", "RPG", "Realistic Vision V1.2", "Samdoesarts Ultmerge", "SD v1.5 inpainting", "SD v1.5", "seek-art-mega-midjrnyv4-dreamlike", "StablyAiffused's Aesthetic v2.6", "SynthwavePunk v2", "SynthwavePunk v3", "Uber Realistic Porn Merge v1.2", "s1dlxBrew"]
    model_3 = "dalcefo_v3_pastelMix"  # @param ["None", "AbyssOrangeMix2_hard", "Anything v3", "Babes", "ComicsBlend", "dalcefo_v3_pastelMix", "Dreamlike Photoreal", "dreamlikesamkuvshinov", "DreamShaper_v3", "F222", "GTM UltimateBlend v3", "HassanBlend v1.5", "Hunter69 v1", "Inkpunk Diffusion v2", "Kotosmix", "NeverEnding Dream", "Novel Inkpunk F222", "openjourney v2", "RPG", "Realistic Vision V1.2", "Samdoesarts Ultmerge", "SD v1.5 inpainting", "SD v1.5", "seek-art-mega-midjrnyv4-dreamlike", "StablyAiffused's Aesthetic v2.6", "SynthwavePunk v2", "SynthwavePunk v3", "Uber Realistic Porn Merge v1.2", "s1dlxBrew"]
    model_4 = "StablyAiffused's Aesthetic v2.6"  # @param ["None", "AbyssOrangeMix2_hard", "Anything v3", "Babes", "ComicsBlend", "dalcefo_v3_pastelMix", "Dreamlike Photoreal", "dreamlikesamkuvshinov", "DreamShaper_v3", "F222", "GTM UltimateBlend v3", "HassanBlend v1.5", "Hunter69 v1", "Inkpunk Diffusion v2", "Kotosmix", "NeverEnding Dream", "Novel Inkpunk F222", "openjourney v2", "RPG", "Realistic Vision V1.2", "Samdoesarts Ultmerge", "SD v1.5 inpainting", "SD v1.5", "seek-art-mega-midjrnyv4-dreamlike", "StablyAiffused's Aesthetic v2.6", "SynthwavePunk v2", "SynthwavePunk v3", "Uber Realistic Porn Merge v1.2", "s1dlxBrew"]
    model_5 = "NeverEnding Dream"  # @param ["None", "AbyssOrangeMix2_hard", "Anything v3", "Babes", "ComicsBlend", "dalcefo_v3_pastelMix", "Dreamlike Photoreal", "dreamlikesamkuvshinov", "DreamShaper_v3", "F222", "GTM UltimateBlend v3", "HassanBlend v1.5", "Hunter69 v1", "Inkpunk Diffusion v2", "Kotosmix", "NeverEnding Dream", "Novel Inkpunk F222", "openjourney v2", "RPG", "Realistic Vision V1.2", "Samdoesarts Ultmerge", "SD v1.5 inpainting", "SD v1.5", "seek-art-mega-midjrnyv4-dreamlike", "StablyAiffused's Aesthetic v2.6", "SynthwavePunk v2", "SynthwavePunk v3", "Uber Realistic Porn Merge v1.2", "s1dlxBrew"]
        
    for model in [model_1, model_2, model_3, model_4, model_5]:
        if model != "None":
            download_models(model)


def download_models(model: str) -> None:
    models = {
        "AbyssOrangeMix2_hard": {
            "site": "hf",
             "sub-url": [
                 "WarriorMama777/OrangeMixs",
                 "Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors",
             ] 
        },
        "Anything v3": {"site": "civitai", "sub-url": "75"},
        "Babes": {"site": "civitai", "sub-url": "8828"},
        "ComicsBlend": {"site": "civitai", "sub-url": "1592"},
        "dalcefo_v3_pastelMix": {"site": "civitai", "sub-url": "6278"},
        "Dreamlike Photoreal": {"site": "civitai", "sub-url": "4224"},
        "DreamShaper_v3": {"site": "civitai", "sub-url": "6202"},
        "dreamlikesamkuvshinov": {"site": "civitai", "sub-url": "1574"},
        "F222": {"site": "civitai", "sub-url": "1224"},
        "GTM UltimateBlend v3": {"site": "civitai", "sub-url": "5618"},
        "HassanBlend v1.5": {"site": "civitai", "sub-url": "4635"},
        "Hunter69 v1": {"site": "civitai", "sub-url": "5349"},
        "Inkpunk Diffusion v2": {"site": "civitai", "sub-url": "1138"},
        "Kotosmix": {"site": "civitai", "sub-url": "6087"},
        "NeverEnding Dream": {"site": "civitai", "sub-url": "11925?type=Pruned%20Model"},
        "Novel Inkpunk F222": {"site": "civitai", "sub-url": "1220"},
        "openjourney v2": {
            "site": "hf",
            "sub-url": ["prompthero/openjourney-v2", "openjourney-v2-unpruned.ckpt"],
        },
        "RPG": {"site": "civitai", "sub-url": "4350"},
        "Realistic Vision V1.2": {"site": "civitai", "sub-url": "6987"},
        "SD v1.5 inpainting": {
            "site": "hf",
            "sub-url": [
                "runwayml/stable-diffusion-inpainting",
                "sd-v1-5-inpainting.ckpt",
            ],
        },
        "SD v1.5": {
            "site": "hf",
            "sub-url": ["runwayml/stable-diffusion-v1-5", "v1-5-pruned-emaonly.ckpt"],
        },
        "seek-art-mega-midjrnyv4-dreamlike": {"site": "civitai", "sub-url": "2066"},
        "StablyAiffused's Aesthetic v2.6": {"site": "civitai", "sub-url": "6266"},
        "Samdoesarts Ultmerge": {"site": "civitai", "sub-url": "77"},
        "SynthwavePunk v2": {"site": "civitai", "sub-url": "1144"},
        "SynthwavePunk v3": {"site": "civitai", "sub-url": "1292"},
        "Uber Realistic Porn Merge v1.2": {"site": "civitai", "sub-url": "4971"},
        "s1dlxBrew": {"site": "civitai", "sub-url": "6980"},
    }
    model_data = models.get(model, {})
    directory_path = "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion"
    if model_data.get("site", "") == "civitai":
        download_from_civitai(model_data.get("sub-url", ""), directory_path)
    elif model_data.get("site", "") == "hf":
        repository = model_data.get("sub-url", "")[0]
        file = model_data.get("sub-url", "")[1]
        download_from_huggingfaces(repository, file, directory_path)

    # download VAEs
    vae_path = "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/VAE"
    
    vaes = [
        [
            "stabilityai/sd-vae-ft-mse-original",
            "vae-ft-mse-840000-ema-pruned.ckpt",
         ],
         [
             "hakurei/waifu-diffusion-v1-4",
              "vae/kl-f8-anime2.ckpt",
        ],
        [
            "andite/anything-v4.0",
            "anything-v4.0.vae.pt",
        ]
    ]
    for repo, file in vaes:
        download_from_huggingfaces(repo, file, vae_path)


get_models()

In [ ]:
#@title <font size="5">Install Extensions and Scripts</font>
extensions_directory = "/content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions"
extensions = [
    # "deforum-art/deforum-for-automatic1111-webui",
    "yfszzx/stable-diffusion-webui-images-browser",
    # "camenduru/stable-diffusion-webui-huggingface",
    "camenduru/sd-civitai-browser",
    "camenduru/sd-webui-additional-networks",
    "Coyote-A/ultimate-upscale-for-automatic1111",
    "arenatemp/stable-diffusion-webui-model-toolkit",
    # "adieyal/sd-dynamic-prompts",
    "AUTOMATIC1111/stable-diffusion-webui-wildcards",
    "innightwolfsleep/stable-diffusion-webui-randomize",
    "mix1009/model-keyword",
]
    
for i in extensions:
    download_from_github(i, extensions_directory)


scripts_directory = "/content/gdrive/MyDrive/sd/stable-diffusion-webui/scripts"
scripts = [
    "https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py",
    "https://raw.githubusercontent.com/ArrowM/auto1111-improved-prompt-matrix/main/improved_prompt_matrix.py",
    "https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py",
]

for i in scripts:
    download_file(i, scripts_directory)

download_file(
    "https://raw.githubusercontent.com/WinPriv/auto1111-improved-prompt-matrix-seedy/main/improved_prompt_matrix.py",
    scripts_directory,
    file="improved_prompt_matrix_seed.py"
)

clear_output()
inf('\u2714 Extensions downloaded.','success', '350px')

In [ ]:
#@title Download Embeddings, Loras and hypernetworks

embeddings_path = "/content/gdrive/MyDrive/sd/stable-diffusion-webui/embeddings"
lora_path = "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora"
hyper_path = "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/hypernetworks"

## Embeddings
huggingface_embeddings = [
    ["Nerfgun3/bad_prompt", "bad_prompt_version2.pt"],
    ["yesyeahvh/bad-hands-5", "bad-hands-5.pt"],
    ["nick-x-hacker/bad-artist", "bad-artist.pt"],
    ["nick-x-hacker/bad-artist", "bad-artist-anime.pt"],
]
for repo, file in huggingface_embeddings:
    download_from_huggingfaces(repo, file, embeddings_path)

## Loras
civitai_lora = [
    '8750', #  Korean Doll Likeness
    '9070', #  Taiwan Doll Likeness
    '5480', #  Hinata Hyuuga
    '7804', #  SamDoesArt
    '5108', #  hyperbreasts
    '7844', #  hyperpreg
    '9934', #  hyperpregnant
    '11827', # boobcrush
    '6279', #  button gap 
    '8613', #  futanari
    '6695', #  style - jelly
    '7817', #  incase
    '11085', # yeji itzy
]
for i in civitai_lora:
  download_from_civitai(i, lora_path)

## Hypernetworks
# download_from_huggingfaces("", "", hyper_path)

clear_output()
inf('\u2714 Loras downloaded.', 'success', '350px')

In [ ]:
# !cp -r /content/drive/MyDrive/sd/models/tmp_mix_v2.1.ckpt /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/tmp_mix.ckpt
# print("done")

In [ ]:
#@markdown # Start Stable-Diffusion
from IPython.utils import capture
import time
import sys
import fileinput

Use_localtunnel = False #@param {type:"boolean"}

User = "genius" #@param {type:"string"}
Password= "artist" #@param {type:"string"}
#@markdown - Add credentials to your Gradio interface (optional).

auth=f"--gradio-auth {User}:{Password}"
if User =="" or Password=="":
  auth=""

with capture.capture_output() as cap:
  if not os.path.exists('/tools/node/bin/lt'):
    !npm install -g localtunnel

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/
  !wget -O paths.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/paths.py
  !wget -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/extras.py
  !wget -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/sd_models.py
  !wget -q -O /usr/local/lib/python3.8/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/

  !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/sd_models.py
  !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py
  !sed -i "s@map_location='cpu'@map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/extras.py

share=''
if not Use_localtunnel:
  share='--share'

else:
  with capture.capture_output() as cap:
    share=''
    %cd /content
    !nohup lt --port 7860 > srv.txt 2>&1 &
    time.sleep(2)
    !grep -o 'https[^ ]*' /content/srv.txt >srvr.txt
    time.sleep(2)
    srv= getoutput('cat /content/srvr.txt')

    for line in fileinput.input('/usr/local/lib/python3.8/dist-packages/gradio/blocks.py', inplace=True):
      if line.strip().startswith('self.server_name ='):
          line = f'            self.server_name = "{srv[8:]}"\n'
      if line.strip().startswith('self.protocol = "https"'):
          line = '            self.protocol = "https"\n'
      if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
          line = ''
      if line.strip().startswith('else "http"'):
          line = ''
      sys.stdout.write(line)
            
    !rm /content/srv.txt /content/srvr.txt
    %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui

try:
  model
  if os.path.isfile(model):
    !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-half-vae  --ckpt "$model" --xformers $auth --disable-console-progressbars
  else:
    !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-half-vae  --ckpt-dir "$model" --xformers $auth --disable-console-progressbars
except:
   !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-half-vae --xformers $auth --disable-console-progressbars

In [ ]:
# !cp -r /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/tmp_mix_v2.ckpt /content/drive/MyDrive/sd/models/tmp_mix.ckpt
# print("done")